# Proyecto análisis de conversación de mensajería

## Cargar datos
- Exportar una conversación personal o grupal de Whastapp o Telegram en un formato adecuado (JSON, CSV, etc), no exportar audios, archivos multimedia, etc
- Importar archivo y convertirlo en dataframe
- Obtener y graficar los siguientes datos. Elegir el gráfico que mejor represente los datos

In [1]:
import pandas as pd
import json

with open('data/result.json','r') as file:
    data = json.load(file)

df = pd.DataFrame(data["messages"])
df

,id,type,date,actor,actor_id,action,title,text,members,from,...,width,height,message_id,edited,forwarded_from,photo,mime_type,duration_seconds,via_bot,inviter
0,1,service,2020-03-06T14:00:30,ROSARIO HACKING - RISECON,channel1297943957,migrate_from_group,ROSARIO HACKING - RISECON,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,service,2020-03-06T16:16:00,Liso,user7840615,invite_members,NaN,,[Liso],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,message,2020-03-06T16:19:34,NaN,NaN,NaN,NaN,[Bienvenidos al grupo Rosario Hacking! Compart...,NaN,Rama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,service,2020-03-06T17:36:35,DIEGO2K,user766820829,invite_members,NaN,,[DIEGO2K],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,message,2020-03-06T17:37:07,NaN,NaN,NaN,NaN,Acá es para hackear facebook?,NaN,DIEGO2K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3563,3631,message,2021-10-04T17:08:12,NaN,NaN,NaN,NaN,"[{'type': 'link', 'text': 'https://twitter.com...",NaN,Nicolás Antonelli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3564,3632,message,2021-10-04T17:09:01,NaN,NaN,NaN,NaN,"[{'type': 'link', 'text': 'https://twitter.com...",NaN,Nicolás Antonelli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3565,3633,message,2021-10-04T17:09:14,NaN,NaN,NaN,NaN,"[{'type': 'italic', 'text': 'oh shit here we g...",NaN,Nicolás Antonelli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3566,3634,message,2021-10-04T17:17:21,NaN,NaN,NaN,NaN,Twitter tambien esta tirando fallas por lo visto,NaN,Sebastián,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cantidad de mensajes enviados por cada usuario

In [2]:
groups = ['from']
df.groupby(groups,as_index=False).agg(messages = ('text',"count"))
#df.groupby('from')[['id']].count()

,from,messages
0,%00,1
1,10,15
2,Adolfo Fioranelli,446
3,AleSan,30
4,Alejandro Guzman,16
...,...,...
74,Wilder,3
75,Zaynetcero Sas De Cv,82
76,guillermoivan,18
77,nikitux,134


### Promedio de palabras utilizadas por mensaje por cada usuario

In [3]:
#Crear una columna nueve en el dataframe contando la cantidad de palabras por mensaje
df['total_words'] = [len(str(x).split(" ")) for x in df['text']]
groups = ['from','total_words']
agregations = {
    'total_words':["min","max","mean"]
}
df.groupby(groups,as_index=False).agg(agregations)
#Agrupar por usuario y cantidad de palabras
df['total_words'] = [len(str(x).split(" ")) for x in df['text']]
df.groupby(['from'])[['total_words']].mean()

,total_words
from,
%00,9.000000
10,8.666667
Adolfo Fioranelli,17.199552
AleSan,3.633333
Alejandro Guzman,6.250000
...,...
Wilder,1.333333
Zaynetcero Sas De Cv,8.182927
guillermoivan,11.888889


### Porcentaje de mensajes enviados por cada usuario

In [6]:
#df.groupby(['from'],as_index=False).agg(
#    avegare=('id', lambda x: (x.count()/df['type'])*100)
#)
df.groupby('from')['type'].count()/df['type'].count()*100

from
%00                      0.028027
10                       0.420404
Adolfo Fioranelli       12.500000
AleSan                   0.840807
Alejandro Guzman         0.448430
                          ...    
Wilder                   0.084081
Zaynetcero Sas De Cv     2.298206
guillermoivan            0.504484
nikitux                  3.755605
taitan                   0.056054
Name: type, Length: 79, dtype: float64

### Frecuencia de envío de mensaje dependiendo la hora del día

In [12]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%dT%H:%M:%S')
df['hour_only'] = pd.to_datetime(df['date'], format='%Y-%m-%dT%H:%M:%S').dt.hour
df.groupby('hour_only')['type'].count()

hour_only
0      40
1      23
2      20
3       7
4       4
5       3
6       4
7      14
8      84
9     246
10    176
11    218
12    283
13    289
14    183
15    227
16    207
17    222
18    254
19    198
20    222
21    266
22    188
23    190
Name: type, dtype: int64

### Frecuencia de envío de mensaje dependiendo del día de la semana

In [11]:
df['date_only'] = [d.date() for d in df['date']]
df['time_only'] = [d.time() for d in df['date']]
groups = [df['date_only'].dt.to_period('D'),"type"]
agregations = {'type':["count"]}

df.groupby(groups).agg(agregations)

AttributeError: Can only use .dt accessor with datetimelike values

### Hora del día preferida por cada usuario para enviar mensajes

In [13]:
number_of_days = (df['date'].max() - df['date'].min()).days
print(number_of_days)
groups = ['from']
agregations = {
        'text': lambda x: (x.count()/24)/number_of_days 
    #(aux[x.dt.hour])
            }
df.groupby( groups).agg(agregations)

577


,text
from,
%00,0.000072
10,0.001083
Adolfo Fioranelli,0.032207
AleSan,0.002166
Alejandro Guzman,0.001155
...,...
Wilder,0.000217
Zaynetcero Sas De Cv,0.005921
guillermoivan,0.001300


### Día de la semana preferida por cada usuario para enviar mensajes

In [14]:
number_of_weeks = ((df['date'].max() - df['date'].min()).days)/7
print(number_of_weeks)
groups = ['from',df['date'].dt.weekday]
print(df['from'].unique())
df_aux = df.groupby( groups).agg(total_messages=('from',lambda x: x.count()/number_of_weeks)
                       )
df_aux.groupby(['date']).agg(total_users = ('total_messages',lambda x: x.sum()/len(df_aux.index.unique())))

82.42857142857143
[nan 'Rama' 'DIEGO2K' 'Rubén Ávalos' 'M M' 'Martin V' 'Federico Lazcano'
 'Liso' 'R R' 'Ulises' 'Javier Zender' 'Marcelo' 'Diego Bruno'
 'Jorge Martín Vila' 'Ariel Dubra' 'Blackguy' 'Nac' 'Adolfo Fioranelli'
 'Gustavo VEGA' 'Carlos' 'Chedus' 'SparK' 'Gabriel' 'Maxi Soler' 'Nelson'
 'Jose Jaimes' None '10' 'Angel' 'nikitux' 'Serjy' 'Elputocarry' 'Javier'
 'Maximiliano' 'Federico' 'Luis' 'Luciano ᕕ( ᐛ )ᕗ 🇦🇷 (ES, EN, EO)'
 'AleSan' 'Nico Tq' 'Nicolás Mayer' 'Blackie' 'Leonel' 'Lucía Fabbri'
 'R4d4buR 🇦🇷' 'taitan' 'Lucas D' 'Amilcar 🇦🇷' 'Nicolás Antonelli'
 'Juan Cruz' 'Wilder' 'Facundo' 'Bernardo Cifre' 'Max' 'Sebastián'
 'Sergio' 'CharlyHackr' 'Zaynetcero Sas De Cv' 'Flower🌸' '%00' 'Mati'
 'Pablo' 'guillermoivan' 'Anto Z' 'Cosmic Castaway' 'Luis A. Segovia'
 'Hector Alvarado' 'Juls' 'Maxi' 'Laru' 'Raybar' 'Cristian (Segu-Info)'
 'Hernan' 'Leo' 'Alejandro Guzman' 'Luis Hidalgo' 'Joaquin Ramiro Goy' 'P'
 'Lucas Ollarce' 'M B' 'Andy' 'Gonzalo Monja']


,total_users
date,
0,0.022409
1,0.014836
2,0.019782
3,0.029518
4,0.023568
5,0.012827
6,0.007573


### Cantidad promedio de mensajes enviados por hora para cada usuario

### Cantidad de usuarios promedio que envían mensaje por cada día de la semana

### Cantidad de mensajes envíados por mes

### Cantidad mínima, máxima y promedio de palabras por mensaje